## Join Operations

In [17]:
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = '5432'
dbname = 'shop_ease_db'
user = 'postgres'
password = '#Post1'

# Create connection to the database
engine = create_engine(f"postgresql+psycopg://{user}:{password}@{host}:{port}/{dbname}")

# Joining orders, products, and customers tables
query = """
SELECT *
FROM orders
NATURAL JOIN products
NATURAL JOIN customers;
"""
df_orders_products_customers = pd.read_sql(query, engine)
df_orders_products_customers.head(10)

# engine.dispose()

,customer_id,product_id,order_id,order_date,quantity,total_revenue,year,month,day,product_name,category,price,customer_name,email,join_date
0,20,3,1,2024-11-01,2,900.0,2024,11,1,Tablet,Electronics,450.0,Tina Young,tina@example.com,2023-12-22
1,36,1,2,2024-11-01,1,900.0,2024,11,1,Laptop,Electronics,1200.0,James Walker,james@example.com,2023-10-09
2,1,8,3,2024-11-01,3,1800.0,2024,11,1,Headphones,Electronics,150.0,Alice Johnson,alice@example.com,2023-01-15
3,36,6,4,2024-11-02,1,1350.0,2024,11,2,Smartwatch,Electronics,200.0,James Walker,james@example.com,2023-10-09
4,14,12,5,2024-11-02,2,1800.0,2024,11,2,Desk,Furniture,300.0,Nina Mitchell,nina@example.com,2023-10-02
5,8,25,6,2024-11-02,4,2250.0,2024,11,2,Oven,Home Appliances,250.0,Hannah Clark,hannah@example.com,2023-06-20
6,18,19,7,2024-11-03,1,1350.0,2024,11,3,Microwave,Home Appliances,150.0,Rachel King,rachel@example.com,2023-12-05
7,47,15,8,2024-11-03,1,900.0,2024,11,3,Sofa,Furniture,500.0,Ursula Clark,ursula@example.com,2023-06-21
8,22,33,9,2024-11-03,3,1350.0,2024,11,3,Juice,Food,2.0,Victoria Gonzalez,victoria@example.com,2023-02-12
9,8,7,10,2024-11-04,2,1800.0,2024,11,4,Camera,Electronics,650.0,Hannah Clark,hannah@example.com,2023-06-20


In [ ]:
-- Joining orders, products, and customers tables

SELECT DISTINCT *
FROM orders AS o
JOIN products AS p
USING(product_id)
JOIN customers AS c
USING(customer_id);


## Use Subqueries

In [24]:
order = "SELECT * FROM orders LIMIT 5;"

product = "SELECT * FROM products LIMIT 5;"

order_item = "SELECT * FROM order_items LIMIT 5;"

customer = "SELECT * FROM customers LIMIT 5;"

display(pd.read_sql(order, engine), pd.read_sql(product, engine), pd.read_sql(order_item, engine), pd.read_sql(customer, engine))

,order_id,customer_id,order_date,product_id,quantity,total_revenue,year,month,day
0,1,20,2024-11-01,3,2,900.0,2024,11,1
1,2,36,2024-11-01,1,1,900.0,2024,11,1
2,3,1,2024-11-01,8,3,1800.0,2024,11,1
3,4,36,2024-11-02,6,1,1350.0,2024,11,2
4,5,14,2024-11-02,12,2,1800.0,2024,11,2


,product_id,product_name,category,price
0,1,Laptop,Electronics,1200.0
1,2,Smartphone,Electronics,800.0
2,3,Tablet,Electronics,450.0
3,4,Wireless Mouse,Electronics,25.0
4,5,Keyboard,Electronics,50.0


,order_detail_id,order_id,quantity,product_id
0,1,1,3,34
1,2,1,2,47
2,3,2,4,10
3,4,2,3,77
4,5,3,2,76


,customer_id,customer_name,email,join_date
0,1,Alice Johnson,alice@example.com,2023-01-15
1,2,Bob Smith,bob@example.com,2023-02-20
2,3,Charlie Davis,charlie@example.com,2023-03-05
3,4,David Lee,david@example.com,2023-03-10
4,5,Eve Parker,eve@example.com,2023-04-01


In [ ]:

-- Subquery to find the top 5 customers with the highest order amount in the last month

SELECT 
    p.product_name,
    SUM(o.quantity) AS total_quantity,
    SUM(o.total_revenue) AS grand_total_revenue
FROM orders AS o
JOIN products AS p
USING(product_id)
WHERE o.month = (
    SELECT MAX(month)
    FROM orders
)
GROUP BY p.product_name
ORDER BY 
    total_quantity DESC, 
    grand_total_revenue DESC
LIMIT 5;





In [26]:
# Subquery to find the top 5 customers with the highest order amount in the last month in Python

highest_sales = """
SELECT 
    p.product_name,
    SUM(o.quantity) AS total_quantity,
    SUM(o.total_revenue) AS grand_total_revenue
FROM orders AS o
JOIN products AS p
USING(product_id)
WHERE o.month = (
    SELECT MAX(month)
    FROM orders
)
GROUP BY p.product_name
ORDER BY 
    total_quantity DESC, 
    grand_total_revenue DESC
LIMIT 5;
"""

df_highest_sales = pd.read_sql(highest_sales, engine)
df_highest_sales


,product_name,total_quantity,grand_total_revenue
0,Jeans,64,40326.5
1,Microwave,59,36577.0
2,Juice,52,9680.0
3,Tea,37,17114.0
4,Camera,32,14667.5


## CASE Statement to Categories Orders

In [ ]:
-- Categories of orders

SELECT 
    *,
    CASE 
        WHEN quantity >= 3 THEN 'High'
        WHEN quantity >= 2 THEN 'Medium'
        ELSE'Low'
    END AS revenue_category

FROM orders;

## Query Optimization

In [ ]:
-- Analyze query and execution plan
EXPLAIN ANALYZE
SELECT 
    *,
    CASE 
        WHEN quantity >= 3 THEN 'High'
        WHEN quantity >= 2 THEN 'Medium'
        ELSE'Low'
    END AS revenue_category

FROM orders;

In [ ]:
-- Indexing
CREATE INDEX orders_quantity_idx ON orders (quantity);



In [ ]:
-- Reanalyze query and execution plan
EXPLAIN ANALYZE
SELECT 
    *,
    CASE 
        WHEN quantity >= 3 THEN 'High'
        WHEN quantity >= 2 THEN 'Medium'
        ELSE'Low'
    END AS revenue_category
FROM orders;